In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import george

In [ ]:
from justice import simulate as sim
from justice import summarize as summ
from justice import visualize as vis
from justice import xform
from justice import lightcurve
from justice.datasets import plasticc_data
from tensorflow.contrib.framework import nest

In [ ]:
lcs = lightcurve.PlasticcDatasetLC.get_lc_by_target("data/plasticc_training_data.db",53)



In [ ]:
singlecurve = lcs[15]

xs = np.array([[t for t in singlecurve.bands[b].time] for b in "ugrizY"]).ravel()
xs = np.concatenate(xs)
xs

In [ ]:
filts={'u':300, 'g':400, 'r': 500, 'i': 600, 'z':700, 'Y':800}

In [ ]:
λs = np.array([[filts[b] for t in singlecurve.bands[b].time] for b in "ugrizY"]).ravel()
print(len(xs))
λs = np.concatenate(λs)

ϕs = np.array([[np.log(ϕ+200000.) for ϕ in singlecurve.bands[b].flux] for b in "ugrizY"]).ravel()
ϕs = np.concatenate(ϕs)
# σs = singlecurve.bands['z'].flux_err

ins = np.array([[x,λ] for (x,λ) in zip(xs,λs)]) 
ϕs.shape
ins.shape

In [ ]:
fig = vis.plot_lcs(singlecurve)

In [ ]:
kernel = george.kernels.Matern52Kernel([200.], ndim=2, axes=0) * george.kernels.ExpSquaredKernel([100.], ndim=2, axes=1)

In [ ]:
gp = george.GP(kernel)

In [ ]:
gp.compute(ins)

In [ ]:
testx = np.linspace(np.min(xs), np.max(xs), 100)
testy = np.linspace(300., 1000., 20)

tests = np.array([[x,y] for x in testx for y in testy])

In [ ]:
tests

In [ ]:
mean, covar = gp.predict(ϕs, tests)

In [ ]:
mean = mean.reshape([len(testx),len(testy)])


In [ ]:
gp.get_parameter_dict()

In [ ]:
plt.matshow(mean.T)

In [ ]:
samps = gp.sample_conditional(ϕs, tests, size=1)

In [ ]:
plt.scatter(tests[:,0], tests[:, 1], c=plt.cm.get_cmap('viridis')(samps))